# MLE of logistic regression model

In [47]:
import sys
sys.path.insert(0,'C:\\code\\python_for_the_financial_economist\\')

# import relevant packages
import numpy as np

# packages for convex optimization
import cvxpy as cp

# for calculating evaluation metrics
from sklearn.metrics import accuracy_score, confusion_matrix

from codelib.visualization.layout import DefaultStyle
DefaultStyle();

The logistic regression model specifies the probability that a binary outcome variable $y$ equal 1 as 

$$
P(y=1 \vert \mathbf{x}) = \frac{1}{1 + e^{-\mathbf{x}^\top \boldsymbol{\beta}}}
$$

where $\boldsymbol{\beta}$ is the vector of coefficients to be estimated. 

Below we simulate data from a logistic model with two explanatory variables. 

In [33]:
# set random seed
np.random.seed(42)

# number of simulations
num_sim = 100

# coefficients
beta0 = -1.0
beta1 = 2.0
beta2 =  -3.0

# generate explanatory variables
x1 = np.random.normal(loc=0, scale=1, size=num_sim)
x2 = np.random.normal(loc=0, scale=1, size=num_sim)

# generate probabilties
probs = 1 / (1 + np.exp(-(beta0 + beta1 * x1 + beta2 * x2)))

# simulate y values
y = np.random.binomial(1, probs, size=num_sim)

## Problem 1

The log-likelihood function is given by

$$
\ell(\boldsymbol{\beta}) = \sum_{i=1}^N [y_i \log \frac{1}{1 + e^{-\mathbf{x}^\top \boldsymbol{\beta}}} + (1 - y_i) \log (1 - \frac{1}{1 + e^{-\mathbf{x}^\top \boldsymbol{\beta}}})]
$$

which is a convex function in $\boldsymbol{\beta}$.

Use the `CVXPY` package to estimate the coefficients by maximizing the log-likelihood function. 

### Solution


In [34]:
"""
Solving with CVXPY
"""

x_mat = np.vstack((x1, x2)).T

# define optimization variables
beta = cp.Variable(2)
intercept = cp.Variable()

# define objective
log_likelihood = cp.sum(cp.multiply(y, x_mat @ beta + intercept) - cp.logistic(x_mat @ beta + intercept))
objective = cp.Maximize(log_likelihood)

# define problem 
problem = cp.Problem(objective)

# solve problem
problem.solve()

# solution 
beta_est = beta.value # <- gives the same solution as the analytical formula
beta_est

array([ 3.30648337, -4.33761439])

In [35]:
# intercept
intercept_est = intercept.value
intercept_est

array(-1.47862693)

## Problem 2

Calculate the predicted probabilities. Classify $y$ based on a  50% threshold. In addition calculate the accuracy score and the confusion matrix (use metrics.accuracy_score and metrics.confusion_matrix from `scikit-learn`).

### Solution

In [36]:
probs_pred = 1 / (1 + np.exp(- (x_mat @ beta_est + intercept_est)))

# Classify based on a threshold (e.g., 0.5)
y_pred = (probs_pred >= 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y, y_pred)
conf_matrix = confusion_matrix(y, y_pred)

print(f"\nAccuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.88
Confusion Matrix:
[[57  4]
 [ 8 31]]


The accuracy score is the fraction of correct predictions. 

We interpret the confusion matrix as follows

<table><tbody>
  <tr>
    <td>True Negative</td>
    <td>False Positive</td>
  </tr>
  <tr>
    <td>False Negative</td>
    <td>True Positive</td>
  </tr>
</tbody>
</table>  </tr>
</tbody>

## Problem 3

To be able to perform variable selection, we can add an L1 penalty to the log-likelihood

$$
\ell_{reg}(\boldsymbol{\beta}) = \ell(\boldsymbol{\beta}) - \lambda \Vert \boldsymbol{\beta} \Vert_1 , \; \; \lambda > 0
$$

Again, estimate the parameters using `CVXPY` and evaluate the model. 

### Solution

In [46]:
"""
Solving with CVXPY
"""

lambda_reg = cp.Parameter(nonneg=True)
lambda_reg.value = 3 # you can change this parameter to change the regularization 

# define optimization variables
beta = cp.Variable(2)
intercept = cp.Variable()

# define objective
log_likelihood = cp.sum(cp.multiply(y, x_mat @ beta + intercept) - cp.logistic(x_mat @ beta + intercept)) - lambda_reg * cp.norm1(beta)
objective = cp.Maximize(log_likelihood)

# define problem 
problem = cp.Problem(objective)

# solve problem
problem.solve()

# solution 
beta_reg_est = beta.value # <- gives the same solution as the analytical formula
beta_reg_est

array([ 1.4772392, -2.2053723])

In [44]:
# intercept
intercept_reg_est = intercept.value
intercept_reg_est

array(-0.80438246)

In [45]:
probs_pred = 1 / (1 + np.exp(- (x_mat @ beta_reg_est + intercept_reg_est)))

# Classify based on a threshold (e.g., 0.5)
y_pred = (probs_pred >= 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y, y_pred)
conf_matrix = confusion_matrix(y, y_pred)

print(f"\nAccuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.9
Confusion Matrix:
[[58  3]
 [ 7 32]]


### Links

Logistic regression model with maximum likelihood: [Statlect.com](https://www.statlect.com/fundamentals-of-statistics/logistic-model-maximum-likelihood)